# Preparation

At first, we load data and fill structures (arrays, dictionaries) and count records

In [1]:
preDictWords, preDictTags  = {}, {}
wordsArray, tagsArray = [], []
counter = 0

data = open("train.ester1.cut.bio", "r")
for record in data:
    if (record != '\n'):
        field = record.split(' ')
        field[1] = field[1][:-1]
        counter += 1
        
        wordsArray.append(field[0])
        tagsArray.append(field[1])
        
        if field[0] in preDictWords:
            preDictWords[field[0]] += 1
        else:
            preDictWords[field[0]] = 1
            
        if field[1] in preDictTags:
            preDictTags[field[1]] += 1
        else:
            preDictTags[field[1]] = 1
            
for word in preDictWords:
    preDictWords[word] /= counter
print("Words added")

for tag in preDictTags:
    preDictTags[tag] /= counter
print("Tags added")

N = len(wordsArray)
print("Total number of records = " + str(N))

Words added
Tags added
Total number of records = 840659


# "Matrixes"
Then we create dictionaries of emission and transition (instead of matrixes, because it looks prettier and mpre interesting to work with)

 


In [2]:
matrixE, matrixT = {}, {}
for i in list(preDictWords.keys()):
    matrixE[i]=[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
print("matrixE dictionary created")

tagsList = list(preDictTags.keys())
for i in tagsList:
    matrixT[i]=[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
print("matrixT dictionary created")
      
for j in range(N):
    matrixE[wordsArray[j]][tagsList.index(tagsArray[j])] += 1
print("matrixE dictionary filled")
      
for i in preDictWords.keys():
    sum = 0.
    for n in range(0,15):
        sum = sum + matrixE[i][n]
    for n in range(0,15):
        if sum != 0:
            matrixE[i][n] /= sum
                       
for i in range(0, N-1):
    matrixT[tagsArray[i]][tagsList.index(tagsArray[i+1])] +=1 
print("matrixT dictionary filled")
        
for i in preDictTags.keys():
    sum = 0.
    for n in range(0,15):
        sum = sum + matrixT[i][n]
    for n in range(0,15):
        if sum != 0:
            matrixT[i][n] /= sum
            
print("matrixE for word 'vingt' = " + str(matrixE["vingt"]))
print("Transtions for tag 'I-time' = " + str(matrixT["I-time"]))

matrixE dictionary created
matrixT dictionary created
matrixE dictionary filled
matrixT dictionary filled
matrixE for word 'vingt' = [0.32694710537276134, 0.001665972511453561, 0.007496876301541024, 0.16826322365680965, 0.248646397334444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.024156601416076635, 0.15951686797167847, 0.059558517284464804, 0.0, 0.003748438150770512]
Transtions for tag 'I-time' = [0.4241329479768786, 0.008360858794384807, 0.0, 0.013728323699421965, 0.5298307184145334, 0.00041288191577208916, 0.0, 0.019611890999174236, 0.0, 0.002064409578860446, 0.0, 0.0012386457473162675, 0.0, 0.0006193228736581337, 0.0]


# Algorithm
Next if definition of Viterbi algorithm function
1. Find 1st word the most probably tag
2. Find probabilities come frome 1st step tag to each another
3. Multiple that by probability of next tag for next word
4. Choose max of probabilities and replace it with 1st step tag
5, Come to 2

In [3]:
def viterbi():
    print(" -- Algorithm work started -- ")
    p, ind, answer = [], [], []
    pChosen, indChosen, wordsMinus = -1, -1, 0
    skippedLastWord = False
    for word in range(N - 1):
        if not wordsArray[word] in matrixE:
            wordsMinus += 1
            continue
        wordE = matrixE[wordsArray[word]]
        wordEMax = 0
        tag = ""

        if pChosen == -384761294194:
            wordEMax = max(wordE)
            tag = tagsList[wordE.index(wordEMax)]
        else:
            wordEMax = pChosen
            tag = tagsList[indChosen]

        for ind2 in range(15):
            p1 = wordEMax * matrixT[tag][ind2]
            
            if not wordsArray[word + 1] in matrixE:
                skippedLastWord = True
                continue
            p2 = p1 * matrixE[wordsArray[word + 1]][ind2]
            p.append(p2)
            ind.append(ind2)
        if skippedLastWord:
            skippedLastWord = False
            answer.append('~~~')
        else:
            pChosen = max(p)
            indChosen = ind[p.index(pChosen)]
            p = []
            ind = []
            answer.append(tagsList[indChosen])
    errors = 0
    for i in range(len(answer)):
           if answer[i] != tagsArray[i]:
                errors += 1
    print("Errors: " + str(errors / N))
    print("Skipped: " + str(wordsMinus / N))
    print(" -- Algorithm work ended -- ")

# Applying
Using

In [4]:
viterbi()

wordsArray = []
tagsArray = []

f = open("test.ester1.cut.bio", "r")
for x in f:
    if (x != '\n'):
        curStr = x.split(' ')
        curStr[1] = curStr[1][:-1]
        
        wordsArray.append(curStr[0])
        tagsArray.append(curStr[1])

N = len(wordsArray)

viterbi()

 -- Algorithm work started -- 
Errors: 0.12676483568248242
Skipped: 0.0
 -- Algorithm work ended -- 
 -- Algorithm work started -- 
Errors: 0.1383687076146329
Skipped: 0.033222211471646335
 -- Algorithm work ended -- 
